In [10]:
import pymc as pm
import numpy as np
import pandas as pd

In [4]:
# Data for the 12 hospitals
y_i = np.array([0, 8, 18, 46, 8, 13, 9, 31, 14, 8, 29, 24])  # Deaths
n_i = np.array([47, 148, 119, 810, 211, 196, 148, 215, 207, 97, 256, 360])  # Surgeries

In [12]:
# Calculate mortality rate for each hospital
mortality_rate = deaths / surgeries

# Create a DataFrame
hospital_data = pd.DataFrame({
    'Hospital': range(1, 13),
    'Deaths': deaths,
    'Surgeries': surgeries,
    'Mortality Rate': mortality_rate
})

# Display the DataFrame
print(hospital_data)

    Hospital  Deaths  Surgeries  Mortality Rate
0          1       0         47        0.000000
1          2       8        148        0.054054
2          3      18        119        0.151261
3          4      46        810        0.056790
4          5       8        211        0.037915
5          6      13        196        0.066327
6          7       9        148        0.060811
7          8      31        215        0.144186
8          9      14        207        0.067633
9         10       8         97        0.082474
10        11      29        256        0.113281
11        12      24        360        0.066667


In [5]:
# Define the model
with pm.Model() as model:
    # Hyperprior for a and b of the Beta distribution
    a = pm.Gamma('a', alpha=2, beta=1)
    b = pm.Gamma('b', alpha=2, beta=1)
    
    # Theta for each hospital, with a shared prior
    theta = pm.Beta('theta', alpha=a, beta=b, shape=len(y_i))
    
    # Likelihood of the observed data
    y = pm.Binomial('y', n=n_i, p=theta, observed=y_i)
    
    # Inference
    trace = pm.sample(2000, tune=1000, return_inferencedata=True)


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a, b, theta]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 140 seconds.


In [6]:
# Summary of the posterior
summary = pm.summary(trace)
print(summary)

            mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
a          0.890  0.286   0.410    1.446      0.004    0.003    5606.0   
b          6.440  2.392   2.315   10.885      0.028    0.020    7211.0   
theta[0]   0.016  0.017   0.000    0.047      0.000    0.000    6541.0   
theta[1]   0.057  0.019   0.024    0.092      0.000    0.000   11778.0   
theta[2]   0.149  0.032   0.093    0.210      0.000    0.000   11513.0   
theta[3]   0.057  0.008   0.042    0.073      0.000    0.000   13908.0   
theta[4]   0.041  0.013   0.017    0.065      0.000    0.000   13496.0   
theta[5]   0.068  0.018   0.036    0.102      0.000    0.000   13693.0   
theta[6]   0.064  0.020   0.031    0.102      0.000    0.000   12274.0   
theta[7]   0.143  0.023   0.101    0.188      0.000    0.000   12250.0   
theta[8]   0.070  0.017   0.039    0.101      0.000    0.000   12447.0   
theta[9]   0.085  0.027   0.036    0.136      0.000    0.000   11289.0   
theta[10]  0.114  0.020   0.078    0.1

In [7]:
# To specifically look at the estimates for hospitals A and H
print("Hospital A - Theta estimate:", np.mean(trace.posterior['theta'][:, :, 0]))
print("Hospital H - Theta estimate:", np.mean(trace.posterior['theta'][:, :, 7]))

Hospital A - Theta estimate: <xarray.DataArray 'theta' ()>
array(0.01611139)
Coordinates:
    theta_dim_0  int32 0
Hospital H - Theta estimate: <xarray.DataArray 'theta' ()>
array(0.14326433)
Coordinates:
    theta_dim_0  int32 7
